## Importar Bibliotecas:

### Pip install

### Bibliotecas

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import requests
import re
import folium
#import wget
import geopy
import ast
import folium
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

import time
import json
#import parser

import pandas as pd
from pandas import json_normalize

from geopy.geocoders import Nominatim

#from bs4 import BeautifulSoup

import sys
sys.setrecursionlimit(2000)

# Show all Columns and Rows
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

print('Libraries imported')

Libraries imported


In [2]:
#path = "C:/Users/renan/OneDrive/Área de Trabalho/Estudos/Infnet/Notebooks/location_snackbar.csv"
#dfv = pd.read_csv(path)

## Extração dos dados do foursquare e localização das estações de metrô.

### Mapeamento das Estações

In [3]:
station_dict = {'Catete' : '-22.925580460005982,-43.1766003091964',
                'Largo do Machado' : '-22.930651090814383,-43.17832974593375',
                'Flamengo' : '-22.93684861832403,-43.17829091934274',
                'Botafogo' : '-22.950622880463147,-43.18405212488933',
                'Cardeal Arco Verde' : '-22.96361994621739,-43.18083122492285',
                'Siqueira Campos' : '-22.966722295911186,-43.18649393180904',
                'Cantagalo' : '-22.976112641984226,-43.192580445299605',
                'General Osório' : '-22.984610664497772,-43.19718307781211'}
station_dict

{'Catete': '-22.925580460005982,-43.1766003091964',
 'Largo do Machado': '-22.930651090814383,-43.17832974593375',
 'Flamengo': '-22.93684861832403,-43.17829091934274',
 'Botafogo': '-22.950622880463147,-43.18405212488933',
 'Cardeal Arco Verde': '-22.96361994621739,-43.18083122492285',
 'Siqueira Campos': '-22.966722295911186,-43.18649393180904',
 'Cantagalo': '-22.976112641984226,-43.192580445299605',
 'General Osório': '-22.984610664497772,-43.19718307781211'}

In [4]:
print(station_dict)

{'Catete': '-22.925580460005982,-43.1766003091964', 'Largo do Machado': '-22.930651090814383,-43.17832974593375', 'Flamengo': '-22.93684861832403,-43.17829091934274', 'Botafogo': '-22.950622880463147,-43.18405212488933', 'Cardeal Arco Verde': '-22.96361994621739,-43.18083122492285', 'Siqueira Campos': '-22.966722295911186,-43.18649393180904', 'Cantagalo': '-22.976112641984226,-43.192580445299605', 'General Osório': '-22.984610664497772,-43.19718307781211'}


### Categoria dos Negócios

In [5]:
# Categorias de Negócios

category = [13002,'Bakery']

'''
Guia das categorias:

https://developer.foursquare.com/docs/categories
'''

## ID Categorias de Negócios

category

[13002, 'Bakery']

### Requisições múltiplas

In [6]:
# Chaves para acesso:

client_id = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
client_secret = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
oauth_token = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
radius = 50
limit = 50
# categories
# latitude
# longitude

In [7]:
def make_request(url):
  headers = {
      "Accept": "application/json",
      "Authorization": "xxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
  }
  response = requests.request("GET", url, headers=headers)
  return response

def get_next_url(response):
  try:
    return re.findall("<(.*?)>", response.headers["Link"])[0]
  except (KeyError, IndexError):
    return None

def get_nearby_venues_data(station_dict):
  json_list = []
  url = f"https://api.foursquare.com/v3/places/search?ll={station_dict}&radius=1500&categories=13002&fields=fsq_id%2Cname%2Cgeocodes%2Clocation%2Ccategories&limit=50"

  while url != None:
    response = make_request(url)
    json_file = response.json()
    json_list.append(json_file)
    
    url = get_next_url(response)
    time.sleep(0.1)
    print(url)
    print(response.headers)

  return json_list

def get_venues_data():
  df_venues = pd.DataFrame()
  for station, lat_lon in station_dict.items():
    response = get_nearby_venues_data(lat_lon)
    df_temp = pd.DataFrame.from_dict(response[0]['results'])
    df_temp['estacao_nome'] = station
    df_temp['estacao_lat_lon'] = lat_lon
    df_venues = pd.concat([df_temp, df_venues], axis=0)
  return df_venues

In [8]:
venues_data = get_venues_data();

https://api.foursquare.com/v3/places/search?cursor=c3I6NTA&radius=1500&ll=-22.925580460005982%2C-43.1766003091964&fields=fsq_id%2Cname%2Cgeocodes%2Clocation%2Ccategories&categories=13002&limit=50
{'Connection': 'keep-alive', 'Content-Length': '4031', 'Content-Type': 'application/json; charset=utf-8', 'Link': '<https://api.foursquare.com/v3/places/search?cursor=c3I6NTA&radius=1500&ll=-22.925580460005982%2C-43.1766003091964&fields=fsq_id%2Cname%2Cgeocodes%2Clocation%2Ccategories&categories=13002&limit=50>; rel="next"', 'Server': 'Finatra', 'X-Fsq-Request-ID': '626f3d59d8319606a5056482', 'X-RateLimit-Path': '/v3/places/search', 'X-RateLimit-Limit': '0', 'X-RateLimit-Remaining': '0', 'content-encoding': 'gzip', 'Accept-Ranges': 'bytes', 'Date': 'Mon, 02 May 2022 02:09:29 GMT', 'Via': '1.1 varnish', 'X-Served-By': 'cache-gig2250030-GIG', 'X-Cache': 'MISS', 'X-Cache-Hits': '0', 'X-Timer': 'S1651457369.735984,VS0,VE822', 'Vary': 'Origin'}
None
{'Connection': 'keep-alive', 'Content-Length': '1

In [9]:
df_request = venues_data

In [10]:
df_request.head(2)

,fsq_id,categories,geocodes,location,name,estacao_nome,estacao_lat_lon
0,4c3c9f7da97bbe9a0cc2fadd,"[{'id': 13002, 'name': 'Bakery', 'icon': {'pre...","{'main': {'latitude': -22.984827, 'longitude':...","{'address': 'Rua Visconde de Piraja, 29', 'add...",La Veronese,General Osório,"-22.984610664497772,-43.19718307781211"
1,4e58ee04d22dfba6f46c10ce,"[{'id': 13002, 'name': 'Bakery', 'icon': {'pre...","{'main': {'latitude': -22.978994, 'longitude':...","{'address': 'Avenida Epitácio Pessoa, 1789', '...",Padaria Santa Marta,General Osório,"-22.984610664497772,-43.19718307781211"


In [11]:
df_request.to_csv('request_padaria.csv')

## Tratamento dos Dados

### Informações dos dados brutos

In [12]:
df_handling = pd.read_csv('C:/Users/renan/OneDrive/Área de Trabalho/Estudos/Infnet/Notebooks/Projeto_z_sul/request_padaria.csv')

In [13]:
df_handling.drop(df_handling.columns[[0]],axis=1, inplace=True)

In [14]:
df_handling.shape

(400, 7)

In [15]:
df_handling.head(1)

,fsq_id,categories,geocodes,location,name,estacao_nome,estacao_lat_lon
0,4c3c9f7da97bbe9a0cc2fadd,"[{'id': 13002, 'name': 'Bakery', 'icon': {'pre...","{'main': {'latitude': -22.984827, 'longitude':...","{'address': 'Rua Visconde de Piraja, 29', 'add...",La Veronese,General Osório,"-22.984610664497772,-43.19718307781211"


#### Coluna Location

In [16]:
# Criar nova coluna com endereços e adicionar ao dataframe

endereço = []

for location in df_handling['location']:
  dict_locations = ast.literal_eval(location)
  endereço.append(dict_locations['formatted_address'])

df_handling['endereço'] = endereço

In [17]:
df_handling.head(1)

,fsq_id,categories,geocodes,location,name,estacao_nome,estacao_lat_lon,endereço
0,4c3c9f7da97bbe9a0cc2fadd,"[{'id': 13002, 'name': 'Bakery', 'icon': {'pre...","{'main': {'latitude': -22.984827, 'longitude':...","{'address': 'Rua Visconde de Piraja, 29', 'add...",La Veronese,General Osório,"-22.984610664497772,-43.19718307781211","Rua Visconde de Piraja, 29, Rio de Janeiro, RJ..."


#### Coluna geocodes

In [18]:
def extract_value(df, column, regex):
    return df[column].str.extract(regex)

In [19]:
df_handling["geocodes"] = extract_value(df_handling, "geocodes", "{'main':.\{(.*?)\}")

In [20]:
df_handling.geocodes.duplicated().value_counts()

True     211
False    189
Name: geocodes, dtype: int64

In [21]:
# Criar colundas com latitude e longitude e adicionar ao dataframe

latitude = []
longitude = []

for geo in df_handling['geocodes'].str.split():
 latitude.append(geo[1][:-1])
 longitude.append(geo[3][:-1])

df_handling['latitude'] = latitude
df_handling['longitude'] = longitude

In [22]:
df_handling.head(2)

,fsq_id,categories,geocodes,location,name,estacao_nome,estacao_lat_lon,endereço,latitude,longitude
0,4c3c9f7da97bbe9a0cc2fadd,"[{'id': 13002, 'name': 'Bakery', 'icon': {'pre...","'latitude': -22.984827, 'longitude': -43.196363","{'address': 'Rua Visconde de Piraja, 29', 'add...",La Veronese,General Osório,"-22.984610664497772,-43.19718307781211","Rua Visconde de Piraja, 29, Rio de Janeiro, RJ...",-22.984827,-43.19636
1,4e58ee04d22dfba6f46c10ce,"[{'id': 13002, 'name': 'Bakery', 'icon': {'pre...","'latitude': -22.978994, 'longitude': -43.200804","{'address': 'Avenida Epitácio Pessoa, 1789', '...",Padaria Santa Marta,General Osório,"-22.984610664497772,-43.19718307781211","Avenida Epitácio Pessoa, 1789, Rio de Janeiro, RJ",-22.978994,-43.20080


In [23]:
df_handling.drop(df_handling.columns[[0,1,2,3]],axis=1, inplace=True)

In [24]:
df_handling.head(1)

,name,estacao_nome,estacao_lat_lon,endereço,latitude,longitude
0,La Veronese,General Osório,"-22.984610664497772,-43.19718307781211","Rua Visconde de Piraja, 29, Rio de Janeiro, RJ...",-22.984827,-43.19636


In [25]:
df_handling.shape

(400, 6)

In [26]:
df_handling.drop_duplicates(inplace=True)
df_handling = df_handling.assign(estabelecimento='padaria')

In [27]:
df_handling.shape

(400, 7)

In [28]:
df_handling.to_csv("handling_padaria.csv")